In [1]:
import sys
import pickle
import gzip
import re
import subprocess
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# Parsing

In [29]:
VocabDict = pd.read_csv('dataset/InstructionDict.csv', index_col=0, names=['ID'])

In [30]:
minizoo_df = pd.read_csv('dataset/minizoo_df.csv')
#minizoo_df = minizoo_df.sample(5000, random_state=54)

In [32]:
import glob
from multiprocessing import Pool
mypool = Pool(processes=40)

outputRoot = '/ws/papers/active/petadroid/code/output/vocab64k/'
inputRoot  = '/ws/mnt/habouch/datasets/zoo_dataset/'

#fileNames     = [fileName.strip() for fileName in open('dataset/minizoo_df.txt', 'r').readlines()]
fileNames     = minizoo_df.sha256.to_list()

doneFileNames = [item.split('/')[-1] for item in glob.glob(outputRoot + '*')]
todoFileNames = np.array(list(set(fileNames).difference(set(doneFileNames))))
print(len(todoFileNames))

50000


In [33]:
mypool.map(parseDex, todoFileNames)
mypool.close();
mypool.join();

# Collect Vocabulary

In [22]:
import numpy as np
import pandas as pd
import glob
import pickle
import gzip

from collections import Counter

outputRoot = '/ws/papers/active/petadroid/code/output/dataframes/'
doneFileNames = [item.split('/')[-1] for item in glob.glob(outputRoot + '*')]

In [2]:
def getArrayList(FileNames: list) -> list:
    arrayList = []
    for fileName in FileNames:
        inputFilePath = outputRoot + fileName
        arrayList.append((fileName, np.array([item.astype('uint8') for item in  pd.read_pickle(inputFilePath, compression='gzip').functionOpcodes.values])))
    return arrayList

from multiprocessing import Pool
mypool = Pool(processes=50)
FileNameGroups = np.array_split(doneFileNames, 50)
result = mypool.map(getArrayList, FileNameGroups)

In [2]:
def buildFunctionInstructionsVocabular(FileNameList : list) -> Counter:
    vocabularCounter = Counter()
    for FileName in FileNameList:
        inputFilePath    = outputRoot + FileName
        vocabularCounter += Counter((' '.join(pd.read_pickle(inputFilePath, compression='gzip').functionInstructions)).split())
        
    return vocabularCounter

from multiprocessing import Pool
mypool = Pool(processes=50)
FileNameGroups = np.array_split(doneFileNames, 50)
result = mypool.map(buildFunctionInstructionsVocabular, FileNameGroups)

instuctionsDict = Counter()
for cnt in result:
    instuctionsDict += cnt

s = pd.DataFrame(list(instuctionsDict.items()), columns=['instruction', 'count'])
s.sort_values('count', inplace=True, ascending=False)
s.reset_index(drop=True, inplace=True)
s.to_csv('dataset/functionInstructionsVocabDict.csv', index=False)

In [ ]:
def buildFunctionMethodCallsVocabular(FileNameList : list) -> Counter:
    vocabularCounter = Counter()
    for FileName in FileNameList:
        inputFilePath    = outputRoot + FileName
        vocabularCounter += Counter((' '.join(pd.read_pickle(inputFilePath, compression='gzip').functionMethodCalls)).split())
        
    return vocabularCounter

from multiprocessing import Pool
mypool = Pool(processes=50)
FileNameGroups = np.array_split(doneFileNames, 50)
result = mypool.map(buildFunctionMethodCallsVocabular, FileNameGroups)

instuctionsDict = Counter()
for cnt in result:
    instuctionsDict += cnt

s = pd.DataFrame(list(instuctionsDict.items()), columns=['instruction', 'count'])
s.sort_values('count', inplace=True, ascending=False)
s.reset_index(drop=True, inplace=True)
s.to_csv('dataset/functionMethodCallsVocabDict.csv', index=False)

In [4]:
def buildFunctionMethodCallsArgsVocabular(FileNameList : list) -> Counter:
    vocabularCounter = Counter()
    for FileName in FileNameList:
        inputFilePath    = outputRoot + FileName
        vocabularCounter += Counter((' '.join(pd.read_pickle(inputFilePath, compression='gzip').functionMethodCallsArgs)).split())
        
    return vocabularCounter

from multiprocessing import Pool
mypool = Pool(processes=50)
FileNameGroups = np.array_split(doneFileNames, 50)
result = mypool.map(buildFunctionMethodCallsArgsVocabular, FileNameGroups)

instuctionsDict = Counter()
for cnt in result:
    instuctionsDict += cnt

s = pd.DataFrame(list(instuctionsDict.items()), columns=['instruction', 'count'])
s.sort_values('count', inplace=True, ascending=False)
s.reset_index(drop=True, inplace=True)
s.to_csv('dataset/functionMethodCallsArgsVocabDict.csv', index=False)

In [3]:
def builldInstructionDictionary(FileName):
    global hexContentRoot
    
    outputFrameFilePath = hexContentRoot + 'frame/'   + FileName
    outputContentFilePath = hexContentRoot + 'content/' + FileName

    inputFilePath = outputRoot + FileName
    df = pd.DataFrame([item[:3] for item in pickle.loads(gzip.open(inputFilePath, 'rb').read())], columns=['className', 'methodName', 'hexContent'])
    df['hexContent'] = df.hexContent.apply(lambda x: ''.join(x.split()))
    hexContent = ''.join(df.hexContent)
    pd.to_msgpack(outputContentFilePath, hexContent, compress='zlib')
    pd.to_msgpack(outputFrameFilePath, df, compress='zlib')

In [4]:
import pandas as pd
import glob
import pickle
import gzip
from multiprocessing import Pool
mypool = Pool(processes=50)

outputRoot = '/ws/papers/active/petadroid/code/output/dataframes/'
#hexContentRoot  = '/ws/papers/active/petadroid_ws/output/hexContent/'
doneFileNames = [item.split('/')[-1] for item in glob.glob(outputRoot + '*')]

In [14]:
len(doneFileNames)

49984

In [7]:
mypool.map(builldInstructionDictionary, doneFileNames)
mypool.join();

In [6]:
import pandas as pd

In [7]:
method_df = pd.read_csv('dataset/functionMethodCallsVocabDict.csv')

### level1 = method_df.instruction.str.split('/').apply(lambda x: x[0]).value_counts()

In [17]:
level2 = method_df.instruction.str.split('/').apply(lambda x: '.'.join(x[:2])).value_counts()